In [2]:
# colab에 KoNLPy 설치

!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [4]:
# 데이터 불러오기

import pandas as pd

data = pd.read_csv('preprocessed_data.csv', encoding='utf-8', index_col=0)

In [5]:
# 데이터 살펴보기

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77149 entries, 0 to 77148
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  70549 non-null  object
dtypes: object(1)
memory usage: 1.2+ MB


In [6]:
from konlpy.tag import Okt

def extract_noun(texts):
  okt = Okt()
  extract_corpus = []
  
  for line in texts:
    if str(line) != "nan": # 비어있는 셀 제외
      temp = okt.pos(str(line), norm=True, stem=True)
      nouns = []
      for i in range(len(temp)):
        if temp[i][1] == "Noun":
          nouns.append(temp[i][0])
      extract_corpus.append(nouns)

  return extract_corpus



In [7]:
# 네이버 영수증 리뷰에서 Noun만 뽑아서 extract_corpus와 비교

extract_corpus = extract_noun(data['Review'])


In [8]:
extract_corpus[:10]

[[],
 [],
 ['꼬끄', '정말', '쫀득', '마카롱', '크기', '맛', '요', '선물', '때', '아주', '것'],
 ['굿'],
 ['오늘', '메뉴', '마카롱', '맛'],
 ['제이', '엠티'],
 ['가성', '비'],
 ['아들'],
 [],
 ['맛', '요']]

In [ ]:
# 맞게 되었는지 이전의 데이터와 비교

print(len(data), len(extract_corpus))

77149 70549


In [ ]:
# Colab에 Khaiii 설치

!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 7.58 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
--

In [ ]:
import csv

reviewList = []
with open('preprocessed_data.csv','r') as f: 
  reader = csv.reader(f)
  for line in reader:
    reviewList.append(line[1])

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

In [1]:
n_tags = ['NNG', 'NNP', 'NNB'] #일반, 고유,의존
x_tags = ['XR']
 

def pos_text(texts):
    #for sent in texts:
    n_tagged = ''
    x_tagged = ''
    nList = []
    xList = []
    for word in api.analyze(texts):
      for morph in word.morphs:
        if morph.tag in n_tags:
          n_tagged = morph.lex
          nList.append(n_tagged)
          
          #n_tagged += morph.lex + '/' + morph.tag + ' '
             
        elif morph.tag in x_tags:
          x_tagged = morph.lex
          xList.append(x_tagged)      
    return nList, xList

import re

nList = []
xList = []
dList = []
#i = 0
def getN(st,en):
  i = 0
  for k in range(st,en):
    if  (reviewList[k]!=''):
      nList,xList=pos_text(reviewList[k])
      dList.append(nList)    
  return dList
  

In [ ]:
dList = getN(1,len(reviewList))


In [ ]:
 extract_corpus_okt = extract_corpus.copy()
 extract_corpus_khaiii = dList.copy()

In [ ]:
print(len(extract_corpus_okt), len(extract_corpus_khaiii))

70549 70549


In [ ]:
def okt_khaiii(okt_corpus, khaiii_corpus):

  cross_data = []

  for okt_review, khaiii_review in zip(okt_corpus, khaiii_corpus):
    temp = []
    for word in okt_review:
      if word in khaiii_review:
        temp.append(word)
      else:
        continue
    if temp != []: # 비어있는 셀은 제외
      cross_data.append(temp)
    else:
      continue

  return cross_data



In [ ]:
cross_data = okt_khaiii(okt_corpus=extract_corpus_okt, khaiii_corpus = extract_corpus_khaiii)
len(cross_data)

53307

In [ ]:
print("Okt\n", extract_corpus_okt[:10])
print("Khaiii\n", extract_corpus_khaiii[:10])
print("Cross\n", cross_data[:10])

Okt
 [[], [], ['꼬끄', '정말', '쫀득', '마카롱', '크기', '맛', '요', '선물', '때', '아주', '것'], ['굿'], ['오늘', '메뉴', '마카롱', '맛'], ['제이', '엠티'], ['가성', '비'], ['아들'], [], ['맛', '요']]
Khaiii
 [['ㅎ', 'ㅎ'], [], ['꼬끄', '마카롱', '크기', '선물', '때', '것'], ['굿'], ['오늘', '메뉴', '마카롱', '맛'], ['제이엠티'], ['가성비'], ['아들'], [], ['맛나']]
Cross
 [['꼬끄', '마카롱', '크기', '선물', '때', '것'], ['굿'], ['오늘', '메뉴', '마카롱', '맛'], ['아들'], ['동네', '최고'], ['완전'], ['완전', '사랑', '솔티드', '진짜', '존'], ['마카롱', '때', '곳'], ['마카롱'], ['집', '근처', '마카롱', '집', '방문', '종류', '직원', '조금']]


In [ ]:
detokenized_doc = []

for i in range(len(cross_data)):
  t = ' '.join(cross_data[i])
  detokenized_doc.append(t)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=3)

vect = vectorizer.fit_transform(detokenized_doc)


In [ ]:
vect.toarray().shape

(53307, 3255)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=6)
lda.fit_transform(vect.toarray())

array([[0.03333335, 0.03334969, 0.6332831 , 0.03333335, 0.03333335,
        0.23336716],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.04168115, 0.0424945 , 0.79024169, 0.04176966, 0.04183245,
        0.04198055],
       ...,
       [0.58333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
        0.08333333],
       [0.08333378, 0.08333333, 0.08333333, 0.58333289, 0.08333333,
        0.08333333],
       [0.38888616, 0.05555559, 0.0555588 , 0.05555556, 0.38888781,
        0.05555607]])

In [ ]:
import numpy as np
lda.components_.sum(axis=1)[:, np.newaxis]

array([[25628.14352942],
       [21810.14457677],
       [22274.23706571],
       [30456.03854082],
       [21276.2490253 ],
       [31593.18726191]])

In [ ]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=6):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda.components_, terms)



Topic 1: [('커피', 9339.12), ('자리', 1218.39), ('주차', 578.19), ('바다', 478.87), ('풍경', 473.16), ('인테리어', 442.0)]
Topic 2: [('가격', 3124.91), ('케이크', 1331.1), ('라떼', 866.49), ('빵집', 788.26), ('경치', 562.24), ('스콘', 492.16)]
Topic 3: [('음료', 2066.19), ('생각', 776.53), ('아이스크림', 739.16), ('케익', 712.16), ('가게', 476.34), ('테이블', 453.82)]
Topic 4: [('분위기', 7744.54), ('카페', 5785.62), ('음료', 913.46), ('디저트', 838.56), ('매장', 493.93), ('공간', 481.48)]
Topic 5: [('맛집', 2730.96), ('진짜', 1439.17), ('최고', 1390.16), ('종류', 1246.5), ('추천', 844.2), ('완전', 736.37)]
Topic 6: [('사람', 2202.94), ('방문', 1241.65), ('직원', 1040.5), ('주문', 874.91), ('사진', 737.31), ('빙수', 703.16)]


In [ ]:
lda.components_[0]

array([ 0.55042484,  0.17123221,  0.16666827, ...,  0.16728112,
        0.16880706, 86.13789423])